In [1]:
!jupyter notebook --version


7.4.7


The first cell in this notebook sometimes helps work around a bug where the global decimal arithmetic context keeps resetting to the default.

If this "fix" still does not work for you (results do not seem to be consistent), then you should switch to using plain Python files and running the file from the command line, or using an editor such as PyCharm, VSCode, etc.

[Bug report (still open): https://github.com/jupyter/notebook/issues/5260]


In the cells below we will **explicitly** set the decimal context whenever we rely on it, instead of assuming any prior state.


## Practice: Decimal Arithmetic Contexts and Rounding

This notebook contains a series of practice problems (with solutions) focused on:

* The global `decimal` context
* Local contexts via `decimal.localcontext`
* Precision (`prec`) and rounding modes
* Best practices when using `Decimal` for financial / high-precision calculations


In [2]:
import decimal
from decimal import Decimal, localcontext

# BEST PRACTICES
# --------------
# * Always create Decimals from strings, not floats (to avoid binary FP artifacts).
# * Prefer localcontext() for temporary changes to precision / rounding.
# * Avoid changing the global context unless you really mean to.
# * Be explicit about the quantization you want when rounding to a fixed number of decimals.


### Problem 1 – Banker's Rounding vs Float `round`

You are given a list of prices as strings:

```python
raw_prices = ["1.005", "2.015", "3.025", "4.035"]
```

1. Implement a function `bankers_round(amount: str, ndigits: int = 2) -> Decimal` that:
   * Interprets `amount` as a decimal number **using `Decimal`**, and
   * Rounds it to `ndigits` decimal places using **banker's rounding** (`ROUND_HALF_EVEN`).
2. Use it to round all the values in `raw_prices` and compare the results to `round(float(...), 2)`.
3. Demonstrate at least one case where the `float` rounding and the decimal banker's rounding disagree.

Below is **one possible solution**.


In [3]:
decimal.getcontext().prec = 28                      # use a reasonably high default precision
decimal.getcontext().rounding = decimal.ROUND_HALF_EVEN

raw_prices = ["1.005", "2.015", "3.025", "4.035"]

def bankers_round(amount: str, ndigits: int = 2) -> Decimal:
    """Round using Decimal with ROUND_HALF_EVEN (banker's rounding).

    amount is parsed from a *string* to avoid float binary artifacts.
    """
    q = Decimal(10) ** -ndigits   # e.g. ndigits=2 -> Decimal('0.01')
    x = Decimal(amount)
    return x.quantize(q, rounding=decimal.ROUND_HALF_EVEN)

# Compare with float rounding
results = []
for s in raw_prices:
    dec_val = bankers_round(s, 2)
    float_val = round(float(s), 2)
    results.append((s, dec_val, float_val))

results


[('1.005', Decimal('1.00'), 1.0),
 ('2.015', Decimal('2.02'), 2.02),
 ('3.025', Decimal('3.02'), 3.02),
 ('4.035', Decimal('4.04'), 4.04)]

You should see at least one element where `Decimal` banker's rounding and `round(float(...), 2)` are different, due to binary floating point representation and different rounding behavior.


### Problem 2 – Implement "Half Up" Rounding Without Changing Global Context

You often need **half up** rounding (e.g. common in financial calculations) instead of banker's rounding.

1. Implement a function `round_half_up(amount: str, ndigits: int = 2) -> Decimal` that:
   * Uses a **local context** (`decimal.localcontext`) to temporarily set `ROUND_HALF_UP`, and
   * Returns the value rounded to `ndigits` decimal places.
2. Verify that calling `round_half_up` **does not** permanently change the global context's rounding mode.
3. Compare `ROUND_HALF_EVEN` vs `ROUND_HALF_UP` for the following values, rounded to 2 decimal places:
   * `"0.125"`, `"0.135"`, `"0.145"`

Below is **one possible solution**.


In [4]:
decimal.getcontext().prec = 28
decimal.getcontext().rounding = decimal.ROUND_HALF_EVEN

def round_half_even(amount: str, ndigits: int = 2) -> Decimal:
    q = Decimal(10) ** -ndigits
    return Decimal(amount).quantize(q, rounding=decimal.ROUND_HALF_EVEN)

def round_half_up(amount: str, ndigits: int = 2) -> Decimal:
    """Round using ROUND_HALF_UP, *without* mutating the global context."""
    q = Decimal(10) ** -ndigits
    with localcontext() as ctx:
        ctx.prec = 28
        ctx.rounding = decimal.ROUND_HALF_UP
        return Decimal(amount).quantize(q)

test_values = ["0.125", "0.135", "0.145"]

comparison = []
for s in test_values:
    comparison.append(
        (s, round_half_even(s, 2), round_half_up(s, 2))
    )

global_rounding_after = decimal.getcontext().rounding

comparison, global_rounding_after


([('0.125', Decimal('0.12'), Decimal('0.13')),
  ('0.135', Decimal('0.14'), Decimal('0.14')),
  ('0.145', Decimal('0.14'), Decimal('0.15'))],
 'ROUND_HALF_EVEN')

Check that `global_rounding_after` is still `ROUND_HALF_EVEN`, confirming that `round_half_up` did not mutate the global context.


### Problem 3 – Precision and Accumulation Error

Consider summing the decimal number `0.1` one thousand times.

1. Implement a function `decimal_sum_0_1(n: int, prec: int) -> Decimal` that:
   * Uses `localcontext()` to set the precision to `prec`, and
   * Returns the sum of `Decimal('0.1')` added `n` times.
2. Compute the sum for `n = 1000` with precision values `prec = 5`, `10`, and `28`.
3. Compare with the same sum using regular floating point arithmetic.

Below is **one possible solution**.


In [5]:
def decimal_sum_0_1(n: int, prec: int) -> Decimal:
    """Return sum(Decimal('0.1') for _ in range(n)) using a local precision."""
    with localcontext() as ctx:
        ctx.prec = prec
        total = Decimal('0.0')
        step = Decimal('0.1')
        for _ in range(n):
            total += step
        return total

n = 1000
precisions = [5, 10, 28]

decimal_results = {p: decimal_sum_0_1(n, p) for p in precisions}
float_result = sum(0.1 for _ in range(n))

decimal_results, float_result


({5: Decimal('100.0'), 10: Decimal('100.0'), 28: Decimal('100.0')}, 100.0)

Observe how the decimal result depends on the chosen precision, and how both decimal and float results compare to the mathematically exact answer (`100.0`).


### Problem 4 – Temporary High Precision for Intermediate Results

Sometimes you need higher precision for intermediate results, but a lower precision is fine for your final answer.

1. Write a function `high_precision_sqrt_2(ndigits: int) -> Decimal` that:
   * Uses a **local context** with high precision (e.g. 50 digits) to compute `sqrt(2)` using `Decimal(2).sqrt()`.
   * Then quantizes the final result to `ndigits` decimal places.
2. Demonstrate calling `high_precision_sqrt_2(10)` and inspect the result.
3. Verify that the global precision is unchanged after calling the function.

Below is **one possible solution**.


In [6]:
decimal.getcontext().prec = 28
decimal.getcontext().rounding = decimal.ROUND_HALF_EVEN

def high_precision_sqrt_2(ndigits: int = 10) -> Decimal:
    """Compute sqrt(2) at high precision but return a rounded result.

    Uses a local context with high precision for intermediate calculations,
    then quantizes to ndigits decimal places.
    """
    q = Decimal(10) ** -ndigits
    with localcontext() as ctx:
        ctx.prec = 50  # high precision for the sqrt calculation
        sqrt2 = Decimal(2).sqrt()
        return sqrt2.quantize(q)

global_prec_before = decimal.getcontext().prec
approx_sqrt2 = high_precision_sqrt_2(10)
global_prec_after = decimal.getcontext().prec

approx_sqrt2, global_prec_before, global_prec_after


(Decimal('1.4142135624'), 28, 28)

The values `global_prec_before` and `global_prec_after` should match, confirming that using `localcontext()` did not leak changes to the global context.


### Problem 5 – A Convenience Context Manager

Although `decimal.localcontext()` is already a context manager, you may want a **semantic wrapper** that clearly expresses intent when reading code.

1. Implement a context manager `temporary_decimal_settings(*, prec=None, rounding=None)` that:
   * Internally uses `localcontext()`.
   * Applies the provided `prec` and/or `rounding` to the local context only.
2. Use it to:
   * Temporarily set precision to `6` and rounding to `ROUND_HALF_UP` while computing `Decimal('123.445')` rounded to 2 decimals.
   * Show that **outside** the `with` block, the global context is unchanged.

Below is **one possible solution** using `contextlib.contextmanager`.


In [7]:
from contextlib import contextmanager

@contextmanager
def temporary_decimal_settings(*, prec=None, rounding=None):
    """Temporarily tweak decimal context settings.

    Example:
        with temporary_decimal_settings(prec=6, rounding=decimal.ROUND_HALF_UP):
            ... # do decimal work here
    """
    with localcontext() as ctx:
        if prec is not None:
            ctx.prec = prec
        if rounding is not None:
            ctx.rounding = rounding
        yield ctx

# Demonstration
decimal.getcontext().prec = 28
decimal.getcontext().rounding = decimal.ROUND_HALF_EVEN

before = (decimal.getcontext().prec, decimal.getcontext().rounding)

with temporary_decimal_settings(prec=6, rounding=decimal.ROUND_HALF_UP) as ctx:
    value = Decimal('123.445').quantize(Decimal('0.01'))
    inside = (ctx.prec, ctx.rounding, value)

after = (decimal.getcontext().prec, decimal.getcontext().rounding)

before, inside, after


((28, 'ROUND_HALF_EVEN'),
 (6, 'ROUND_HALF_UP', Decimal('123.45')),
 (28, 'ROUND_HALF_EVEN'))

You should see that:

* The value inside the context is rounded using `ROUND_HALF_UP` with precision 6.
* The global context settings (`before` vs `after`) remain identical.
